In [ ]:
# Convet labeled data in JARVIS to the Lightning Pose format for multiple camera views
# 
# Lightening Pose provides a function to convert DLC labeled data to LP labeled data
# Reference
# https://github.com/danbider/lightning-pose/blob/main/scripts/converters/dlc2lp.py
# 

In [11]:
import glob
import os
import shutil

import numpy as np
import pandas as pd
from PIL import Image

In [12]:
jarvis_dir = r'/home/yiting/Documents/Jarvis_projects/Datasets/annotations/6cam_dataset_231216' # path to the labeled dataset 
lp_dir = r'/home/yiting/Documents/LP_projects/LP_240719' # path to the lp project

# find all labeled data in JARVIS project
prefix_year = '2023' # Trial names starts with XXXX year. 
dirs = [filename for filename in os.listdir(jarvis_dir) if filename.startswith(prefix_year)]
dirs.sort()


cameras = os.listdir(os.path.join(jarvis_dir, dirs[0]))

In [8]:
for c in cameras:
    dfs = []
    for d in dirs: # videos for each trial
        csv_file = glob.glob(os.path.join(jarvis_dir, d, c, "annotations.csv"))[0]

        df1 = pd.read_csv(csv_file, on_bad_lines='skip', header = None, index_col=0) 
        df2 = pd.read_csv(csv_file, skiprows=4, header = None, index_col=0) 
        last_column = df2.shape[1]
        df2 = df2.drop(columns=[last_column],axis='columns')

        # Remove entities row
        isNotEntities = [x != 'entities' for x in df1.index.values]
        df1 = df1.iloc[isNotEntities]
        # Find coords row and remove state columns
        isCoords = [x == 'coords' for x in df1.index.values]
        isXY = [s != 'state' for s in df1.iloc[isCoords].values]
        df1 = df1.iloc[:,isXY[0]]
        df2 = df2.iloc[:, isXY[0]]

        # Replace image file name with its file path
        imgs = list(df2.index.values)
        # Change .jpg to .png (JARVIS- .jpg, LP/DLC- .png)
        im_idx = [i[6:len(i)-4] for i in imgs]
        imgs_new =['img' + format(int(i), '04d') + ".png" for i in im_idx]
        trialname_parts = d.split('_')
        new_vid_name = trialname_parts[0] + 'T' + trialname_parts[1]
        new_col = [f"labeled-data/{new_vid_name}/{c}/{i}" for i in imgs_new]
        df2.index = new_col

        df_tmp = pd.concat([df1,df2])
        
        df_tmp.to_csv(os.path.join(jarvis_dir,d,c, "CollectedData.csv"), header = False)
        df = pd.read_csv(os.path.join(jarvis_dir,d,c, "CollectedData.csv"), header = [0,1,2], index_col=0)
        
        dfs.append(df)
    df_all = pd.concat(dfs)

    os.makedirs(lp_dir, exist_ok=True)

    # save concatenated labels for each camera view
    df_all.to_csv(os.path.join(lp_dir, c + ".csv"))

In [9]:
# Copy videos and labeled frames to the LP project folder
src_vid_dir = r'/home/yiting/Documents/Data/Videos'
os.makedirs(os.path.join(lp_dir,'videos'), exist_ok=True)
for c in cameras: # camera view name
    for d in dirs: # trial video name
        trialname_parts = d.split('_')
        new_vid_name = trialname_parts[0] + 'T' + trialname_parts[1]
        os.makedirs(os.path.join(lp_dir,"labeled-data",new_vid_name, c), exist_ok=True)
        # Convert .jpg to .png and copy frames over
        imgs = [im for im in os.listdir(os.path.join(jarvis_dir,d,c)) if im.endswith('.jpg')]
        for i in imgs:
            im = Image.open(os.path.join(jarvis_dir,d,c,i))
            im_idx = i[6:len(i)-4] 
            new_frame_name ='img' + format(int(im_idx), '04d') + ".png" 
            im.save(os.path.join(lp_dir,"labeled-data",new_vid_name,c, new_frame_name))
        # Copy videos over
        session = d[0:10]
        src = os.path.join(src_vid_dir, session, d, c + '.mp4')
        dst = os.path.join(lp_dir, "videos", new_vid_name + '_' + c + '.mp4')
        shutil.copy(src, dst)



In [10]:
# check if image paths are correct
for c in cameras:
    csv_file = os.path.join(lp_dir, c + ".csv")
    df_all = pd.read_csv(csv_file, header = [0,1,2], index_col=0)
    for im in df_all.index:
        assert os.path.exists(os.path.join(lp_dir, im))